##### Copyright 2020 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Введение в градиенты и автоматическое дифференцирование

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/autodiff"><img src="https://www.tensorflow.org/images/tf_logo_32px.png"> Посмотреть на TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/autodiff.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png"> Запустить в Google Colab</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/autodiff.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"> Посмотреть источник на GitHub</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/autodiff.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png"> Скачать блокнот</a></td>
</table>

## Автоматическое дифференцирование и градиенты

[Автоматическое дифференцирование](https://en.wikipedia.org/wiki/Automatic_differentiation) полезно для реализации алгоритмов машинного обучения, таких как [обратное распространение](https://en.wikipedia.org/wiki/Backpropagation) для обучения нейронных сетей.

В этом руководстве мы обсудим способы вычисления градиентов с помощью TensorFlow, особенно в стремительном исполнении.

## Настроить

In [0]:
import tensorflow as tf

## Вычисление градиентов

To differentiate automatically, TensorFlow needs to remember what operations happen in what order during the *forward* pass.  Then, during the *backward pass*, TensorFlow traverses this list of operations in reverse order to compute gradients.

## Градиентные ленты

TensorFlow предоставляет API [tf.GradientTape](https://www.tensorflow.org/api_docs/python/tf/GradientTape) для автоматической дифференциации; то есть вычисление градиента вычисления относительно его входных переменных. TensorFlow «записывает» все операции, выполненные в контексте `tf.GradientTape` на «ленту». Затем TensorFlow использует эту ленту и градиенты, связанные с каждой записанной операцией, для вычисления градиентов «записанного» вычисления с использованием [дифференцирования в обратном режиме](https://en.wikipedia.org/wiki/Automatic_differentiation) .

Со скалярами:

In [0]:
x = tf.constant(3.0)
# y = x ^ 2
with tf.GradientTape() as t:
  t.watch(x)
  y = x * x
# dy = 2x
dy_dx = t.gradient(y, x)
dy_dx.numpy()

Используя матрицы:

In [0]:
x = tf.constant([3.0, 3.0])

with tf.GradientTape() as t:
  t.watch(x)
  z = tf.multiply(x, x)

print(z)

# Find derivative of z with respect to the original input tensor x
print(t.gradient(z, x))

Все операции с `tf.Variable` добавляются на ленту. Чтобы записывать градиенты относительно входов с постоянным тензором (как указано выше), вам нужно добавить `t.watch(x)` чтобы лента градиента отслеживала входной тензор.

Примечание: общая проблема заключается в том, что входные данные не являются `tf.Tensors` , а представляют собой некоторую тензорную структуру, такую как NumPy `ndarray` или список Python. Входные данные такого типа будут преобразованы в тензоры в первой операции, но не начинаются как тензоры, что может привести к путанице в градиентах `None` или ошибках, поскольку вы случайно передали целые числа (которые не могут образовывать градиенты). Чтобы предотвратить эту проблему, используйте `tf.constant` с соответствующим `dtype` или `tf.data.from_tensor_slices` для преобразования ваших входных данных в тензоры или `tf.data.Dataset` .

### Промежуточные результаты

Вы также можете запросить градиенты выходных данных относительно промежуточных значений, вычисленных во время «записанного» контекста `tf.GradientTape` .

In [0]:
x = tf.constant([3.0, 3.0])

with tf.GradientTape() as t:
  t.watch(x)
  y = tf.multiply(x, x)
  z = tf.multiply(y, y)

# Use the tape to compute the derivative of z with respect to the
# intermediate value y.
# dz_dy = 2 * y, where y = x ^ 2
print(t.gradient(z, y))

Градиентные ленты автоматически просматривают любую переменную, к которой обращаются в пределах их видимости. Вы можете перечислить их в порядке их создания.

In [0]:
x = tf.Variable(2.0)
y = tf.Variable(3.0)

with tf.GradientTape() as t:
  # Don't need any calls to watch(), as when variables x and y
  # get used, they will be automatically watched
  y_sq = y * y
  z = x + y_sq

t.watched_variables()

By default, the resources held by a `GradientTape` are released as soon as `GradientTape.gradient()` method is called. To compute multiple gradients over the same computation, create a `persistent` gradient tape. This allows multiple calls to the `gradient()` method as resources are released when the tape object is garbage collected. For example:

In [0]:
x = tf.constant(3.0)
with tf.GradientTape(persistent=True) as t:
  t.watch(x)
  y = x * x
  z = y * y
print(t.gradient(z, x))  # 108.0 (4*x^3 at x = 3)
print(t.gradient(y, x))  # 6.0 (2 * x)
del t  # Drop the reference to the tape

### Примечания по производительности

- Есть небольшие накладные расходы, связанные с выполнением операций в контексте градиентной ленты. Для большинства стремящихся к исполнению это не будет заметной стоимостью, но вы все равно должны использовать контекст ленты вокруг областей, только там, где это требуется.

- Градиентные ленты используют память для хранения промежуточных результатов, включая входы и выходы, для использования во время обратного прохода.

    Для эффективности некоторым операциям (например, `ReLU` ) не нужно сохранять промежуточные результаты, и они сокращаются во время прямого прохода. Однако, если вы используете `persistent=True` на вашей ленте, *ничего не удаляется,* и ваше пиковое использование памяти будет выше.

### Запись управления потоком

Поскольку ленты записывают операции по мере их выполнения, поток управления Python (с использованием `if` s и `while` s) обрабатывается естественным образом:

In [0]:
def f(x, y):
  output = 1.0
  for i in range(y):
    if i > 1 and i < 5:
      output *= x  # tf.multiply(output, x)
  return output

def grad(x, y):
  with tf.GradientTape() as t:
    t.watch(x)
    out = f(x, y)
  return t.gradient(out, x)

x = tf.constant(2.0)

print(grad(x, 6).numpy())  # 12.0
print(grad(x, 5).numpy())  # 12.0
print(grad(x, 4).numpy())  # 4.0


### Получение градиента `None`

Когда градиент недоступен по какой-либо причине, вы обычно получаете градиент `None` .

Распространенной ситуацией является попытка получить градиент не плавания и не сложного значения.

In [0]:
with tf.GradientTape() as tape:
  x = tf.Variable([[2, 2], [2, 2]], dtype=tf.int8)
  y = x * x
print(tape.gradient(y, x))

In other situations, you may be doing operations that are not differentiable. `DecodeJpeg`, for example, does not produce a gradient.  To see which operations have gradients registered for them, see the list of [raw ops](https://www.tensorflow.org/api_docs/python/tf/raw_ops).

Наконец, вы можете просто получать градиенты, где нет никакой связи между входом и выходом.

In [0]:
x = tf.Variable(2.)
y = tf.Variable(3.)

with tf.GradientTape() as tape:
  z = y * y
print(tape.gradient(x, y))

Может быть удобно вернуть 0 вместо `None` . Вы можете решить, что возвращать, когда у вас есть неподключенные градиенты, используя аргумент `unconnected_gradient` .

In [0]:
x = tf.Variable(2.)
y = tf.Variable(3.)

with tf.GradientTape() as tape:
  z = y * y
print(tape.gradient(x, y, unconnected_gradients=tf.UnconnectedGradients.ZERO))

## Пользовательские градиенты

В некоторых случаях вы можете захотеть контролировать, как именно рассчитываются градиенты, а не использовать значения по умолчанию. Эти ситуации включают в себя:

- Там нет определенного градиента для новой операции, которую вы пишете.
- Расчеты по умолчанию численно нестабильны.
- Вы хотите кэшировать дорогостоящие вычисления с прямого прохода.
- Вы хотите изменить значение (например, используя: `tf.clip_by_value` , `tf.math.round` ) без изменения градиента.

Для написания новой операции вы можете использовать `tf.RegisterGradient` чтобы создать свою собственную. Смотрите эту страницу для деталей. (Обратите внимание, что реестр градиентов является глобальным, поэтому изменяйте его с осторожностью.)

Для последних двух случаев вы можете использовать `tf.custom_gradient` . Вот пример, который применяет `tf.clip_by_norm` к градиенту.


In [0]:
# Establish an identity operation, but clip during the gradient pass
@tf.custom_gradient
def clip_gradients(y):
  def backward(dy):
    return tf.clip_by_norm(dy, 0.5)
  return y, backward

v = tf.Variable(2.0)
with tf.GradientTape() as t:
  output = clip_gradients(v * v)
print(t.gradient(output, v))  # calls "backward", which clips 4 to 2


Смотрите декоратор `tf.custom_gradient` для более подробной информации.

### Градиенты высших порядков

Операции внутри контекстного менеджера `GradientTape` записываются для автоматической дифференциации. Если градиенты вычисляются в этом контексте, то вычисление градиента также записывается. В результате точно такой же API работает и для градиентов более высокого порядка. Например:

In [0]:
x = tf.Variable(1.0)  # Create a Tensorflow variable initialized to 1.0

with tf.GradientTape() as t:
  with tf.GradientTape() as t2:
    y = x * x * x

  # Compute the gradient inside the 't' context manager
  # which means the gradient computation is differentiable as well.
  dy_dx = t2.gradient(y, x)
d2y_dx2 = t.gradient(dy_dx, x)

print("dy_dx:", dy_dx.numpy())  # 3.0
print("d2y_dx2:", d2y_dx2.numpy())  # 6.0

## Расширенные функции для `GradientTape`

В этом разделе мы рассмотрим некоторые менее распространенные способы использования `GradientTape` .

### Управление градиентной записью

Вы можете выбрать просмотр только отдельных переменных. Для этого вы можете отключить просмотр всех переменных по умолчанию. В этом случае вам необходимо будет просмотреть все переменные вручную.

In [0]:
x = tf.Variable(2.0)
y = tf.Variable(3.0)

with tf.GradientTape(watch_accessed_variables=False, persistent=True) as t:
  # Only watch y
  t.watch(y)
  y_sq = y * y
  z = x + y_sq

# Gradient will be None, as x is not watched, so the tape cannot
# differentiate it
print("Gradient with respect to x:", t.gradient(z, x))
# Gradient will be 6, as y is watched
print("Gradient with respect to y:", t.gradient(z, y))
del t

Если вы хотите остановить запись градиентов, вы можете использовать `stop_recording()` чтобы временно приостановить запись.

Если вы хотите начать все сначала, используйте `reset()` . Простой выход из блока градиентной ленты и перезапуск обычно проще для чтения, но вы можете использовать `reset` если выход из блока ленты затруднен или невозможен.

In [0]:
x = tf.Variable(2.0)
y = tf.Variable(3.0)

with tf.GradientTape(persistent=True) as t:
  y_sq = y * y
  # Stop recording so we can calculate an intermediate gradient
  with t.stop_recording():
    print("y with respect to y_sq:", t.gradient(y_sq, y))  # Will be 1
  z = x + y_sq
  t.reset()

print("z with respect to y after reset:", t.gradient(z, y))  # None